In [1]:
// C++ xeus-cling kernel installation under conda:
//
// Step 1:
//
// conda create -n cling
// source activate cling
//
// Step 2:
// conda install -c conda-forge xeus-cling
// ----------------------------------------------------
// Fixing problem: ValueError: '' does not appear to be an IPv4 or IPv6 address
//
// $HOME/anaconda3/lib/python3.7/site-packages/notebook/notebookapp.py
//    def _valdate_ip(self, proposal):
//        value = proposal['value']
//        if value == u'*':
//            value = u'0.0.0.0' # This line 
//        return value
//
//

// Library paths
#pragma cling add_library_path("../obj")
#pragma cling add_library_path("$HOME/local/HEPMC3/lib")
#pragma cling add_library_path("$HOME/local/LHAPDF/lib")
#pragma cling add_library_path("$HOME/local/root/lib")

// Include paths
#pragma cling add_include_path("../include")
#pragma cling add_include_path("../libs")
#pragma cling add_include_path("$HOME/local/HEPMC3/include")
#pragma cling add_include_path("$HOME/local/LHAPDF/include")
#pragma cling add_include_path("$HOME/local/root/include")

// Load shared libraries
#pragma cling load("libHepMC3.so")

// Use C++14 kernel!!!
#pragma cling load("libCore.so")
#pragma cling load("libHist.so")
#pragma cling load("libGpad.so")
#pragma cling load("libRIO.so")
/*
#pragma cling load("libNet.so")
#pragma cling load("libGraf.so")
#pragma cling load("libGraf3d.so")
#pragma cling load("libTree.so")
#pragma cling load("libRint.so")
#pragma cling load("libPostscript.so")
#pragma cling load("libMatrix.so")
#pragma cling load("libPhysics.so")
#pragma cling load("libMathCore.so")
#pragma cling load("libThread.so")
#pragma cling load("libGui.so")
#pragma cling load("libRooFit.so")
#pragma cling load("libMinuit.so")
*/

// Note that ROOT drawing commands do not work
// --------------------------------------------------------------------------

#include <iostream>

// ROOT
#include "TCanvas.h"
#include "TH1.h"

// HepMC3
#include "HepMC3/FourVector.h"
#include "HepMC3/GenEvent.h"
#include "HepMC3/GenParticle.h"
#include "HepMC3/GenVertex.h"
#include "HepMC3/LHEFAttributes.h"
#include "HepMC3/Print.h"
#include "HepMC3/ReaderAscii.h"
#include "HepMC3/Relatives.h"
#include "HepMC3/Selector.h"
#include "HepMC3/WriterAscii.h"

// Own
#include "Graniitti/MH1.h"


std::cout << "Hello world" << std::endl;


Hello world


In [2]:
void ReadEvents(const std::string& inputfile, std::vector<std::vector<double>>& S) {

  // Input and output
  HepMC3::ReaderAscii input(inputfile);

  int events = 0;

  // HepMC3 event object
  HepMC3::GenEvent ev(HepMC3::Units::GEV, HepMC3::Units::MM);

  // Event loop over all events in HepMC3 file
  while (!input.failed()) {
      
    input.read_event(ev);
    if (input.failed()) break;

    // Event vector
    std::vector<double> x;

    // Loop over all particles
    for (HepMC3::ConstGenParticlePtr p1 : ev.particles()) {

      // Check is final state and pion
      if (p1->status() == 1 && std::abs(p1->pid()) < 1000) {

        // Take 4-momentum
        HepMC3::FourVector pvec = p1->momentum();
          
        x.push_back(pvec.e());
        x.push_back(pvec.px());
        x.push_back(pvec.py());
        x.push_back(pvec.pz());
      }
    }
    
    S.push_back(x);
    ++events;

    if (events % 1000 == 0) {
      std::cout << events << " events processed" << std::endl;
    }
  }
  
  std::cout << "Found from input: '" << inputfile << "' " << events << " events" << std::endl;
}

In [3]:
std::string inputfile1 = "../output/test.hepmc3";
std::vector<std::vector<double>> S1;


In [4]:
ReadEvents(inputfile1, S1);


Found from input: '../output/test.hepmc3' 100 events


In [5]:
// Loop over events
TH1D* h1 = new TH1D("blaa", "test", 100, 0.0, 5.0);


In [6]:
for (std::size_t i = 0; i < S1.size(); ++i) {
    h1->Fill( S1[i].at(1) );
}


In [7]:
std::cout << h1->GetMean() << std::endl;



0.29399


@0x7f33e911e460